In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# df = spark.read.format("delta") \
#     .load("/Volumes/workspace/bronze/bronzevolume/bookings/data/")

# display(df)

In [0]:
# df = df.withColumn("amount", col("amount").cast(DoubleType())) \
#     .withColumn("modifiedDate",current_timestamp()) \
#     .withColumn("booking_date", to_date(col("booking_date"))) \
#     .drop("_rescued_data")

# display(df)


In [0]:
df = spark.read.format("delta") \
    .load("/Volumes/workspace/bronze/bronzevolume/flights/data/")

df = df.withColumn("flight_date", to_date(col("flight_date"))) \
    .drop("_rescued_data") \
    .withColumn("modifiedDate",current_timestamp())

display(df)

In [0]:
df = spark.read.format("delta") \
    .load("/Volumes/workspace/bronze/bronzevolume/customers/data/")

df = df.drop("_rescued_data") \
        .withColumn("modifiedDate",current_timestamp())

display(df)

In [0]:
df = spark.read.format("delta") \
    .load("/Volumes/workspace/bronze/bronzevolume/airports/data/")

df = df.drop("_rescued_data") \
    .withColumn("modifiedDate",current_timestamp())

display(df)

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
@dlt.table(
    name = "stage_bookings"
)
def stage_bookings():

    df = spark.readStream.format("delta") \
        .load("/Volumes/workspace/bronze/bronzevolume/bookings/data/")

    return df

In [0]:
@dlt.view(
    name = "trans_bookings"
)

def trans_bookings():
    df = spark.readStream.table("stage_bookings")

    df = df.withColumn("amount", col("amount").cast(DoubleType())) \
        .withColumn("modifiedDate",current_timestamp()) \
        .withColumn("booking_date", to_date(col("booking_date"))) \
        .drop("_rescued_data")

    return df

In [0]:
rules = {
    "rule1": "booking_id is NOT NULL",
    "rule2": "passenger_id is NOT NULL",
    "rule3": "flight_id is NOT NULL",
    "rule4": "airport_id is NOT NULL",
    "rule5": "amount is NOT NULL",
    "rule6": "booking_date is NOT NULL"
}

In [0]:
@dlt.table(
    name = "silver_bookings"
)

@dlt.expect_all_or_drop(rules)
def silver_bookings():
    df = spark.readstream.table("trans_bookings")
    return df

Verify data after pipeline run in silver layer

In [0]:
%sql
select * from workspace.silver.silver_airports

In [0]:
tables = spark.sql("SHOW TABLES IN workspace.default").toPandas()["tableName"]
for t in tables:
    spark.sql(f"DROP TABLE IF EXISTS workspace.default.{t}")